In [1]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('data\gemstone.csv')
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [5]:
df = df.drop('id',axis=1)

In [7]:
x = df.drop('price',axis=1)
y = df[['price']]

In [8]:
categorical_columns = x.select_dtypes(include='O').columns
numerical_columns = x.select_dtypes(exclude='O').columns

In [13]:
cut_categories = ['Fair','Good','Very Good','Premium','Ideal']
color_categories = ['D','E','F','G','H','I','J']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']


In [14]:
from sklearn.impute import SimpleImputer  # handling missing values
from sklearn.preprocessing import StandardScaler  # handling featuer scaling
from sklearn.preprocessing import OrdinalEncoder   # ordinal encoding
from sklearn.pipeline import Pipeline    # pipeline
from sklearn.compose import ColumnTransformer

In [15]:
# Numerical Pipeline

num_pipeline = Pipeline(
    steps=[
        ('SimpleImputer',SimpleImputer(strategy='median')),
        ('StandardScaler',StandardScaler())
    ]
)

# Categorical Pipeline

cat_pipeline = Pipeline(
    steps=[
        ('SimpleImputer',SimpleImputer(strategy='most_frequent')),
        ('OrdinalEncoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories]))
    ]
)

# combine num_pipeline and cat_pipeline

preprocessor = ColumnTransformer([
    ('num_pipeline',num_pipeline,numerical_columns),
    ('categorical_pipeline',cat_pipeline,categorical_columns)
])

In [16]:
# train test split

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=2)

In [17]:
preprocessor.fit_transform(x_train)

array([[ 2.63835074,  0.16747242,  1.44299702, ...,  2.        ,
         5.        ,  4.        ],
       [ 0.25918983, -1.39841035, -0.11992378, ...,  2.        ,
         0.        ,  1.        ],
       [-0.17338488, -2.50373936,  0.92202342, ...,  4.        ,
         0.        ,  7.        ],
       ...,
       [-0.58433086, -1.3062996 ,  3.52689142, ...,  1.        ,
         1.        ,  2.        ],
       [-0.47618718,  0.44380468, -1.68284458, ...,  4.        ,
         5.        ,  4.        ],
       [ 0.71339328, -0.20097058, -0.64089738, ...,  4.        ,
         2.        ,  6.        ]])

In [19]:
x_train = pd.DataFrame(preprocessor.fit_transform(x_train),columns=preprocessor.get_feature_names_out())
x_test = pd.DataFrame(preprocessor.transform(x_test),columns=preprocessor.get_feature_names_out())

In [20]:
# Model Training

from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [21]:
reg = LinearRegression()
reg.fit(x_train,y_train)

LinearRegression()

In [22]:
y_pred = reg.predict(x_test)

In [23]:
reg.coef_

array([[ 6414.8782891 ,   -90.86278285,   -69.24829834, -1465.67822003,
         -280.48563465,  -523.30756715,    70.66068198,  -284.43543394,
          432.56651268]])

In [24]:
reg.intercept_

array([3170.09421763])

In [25]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, mse, rmse, r2_square

In [26]:
models = {
    'LinearRegression':LinearRegression(),
    'Ridge':Ridge(),
    'Lasso': Lasso(),
    'ElasticNet':ElasticNet()
}

model_list = []
r2_list = []


for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(x_train,y_train)
    y_pred = model.predict(x_test)

    mae,mse,rmse,r2_square = evaluate_model(y_pred,y_test)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])


    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)

    r2_list.append(r2_square)
    
    print('='*35)
    print('\n')

LinearRegression
Model Training Performance
RMSE: 1021.3909750763786
MAE: 676.0498840896317
R2 score 93.18922048115965


Ridge
Model Training Performance
RMSE: 1021.4146070008118
MAE: 676.0835977803165
R2 score 93.18863068356715


Lasso
Model Training Performance
RMSE: 1019.3107151775905
MAE: 677.0450443378966
R2 score 93.20718064304941


ElasticNet
Model Training Performance
RMSE: 1533.3269828280877
MAE: 1061.2767342827121
R2 score 78.22219807132696


